In [11]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_selection import RFE, RFECV
import Feature_Importance_Funs

diabetes = pd.read_csv("diabetes.csv")
diabetes.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,1.0.2,2.0.1,3.0.1,4.0.1,5.0.1,6.0.1,7.0,8.0,feature_3,healthy_food
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,5.0,1.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1,0,0,0,0,0,0,0,0.0,0.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,5.0,1.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,1,0,0,2.0,2.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,0,0,0,1,0,0,0,0,2.0,2.0


In [12]:
diabetes['Diabetes_012'] = diabetes['Diabetes_012'].astype(int)
diabetes['Diabetes_012']

0         0
1         0
2         0
3         0
4         0
         ..
253675    0
253676    2
253677    0
253678    0
253679    2
Name: Diabetes_012, Length: 253680, dtype: int64

In [13]:
# Defining the input and target variables
X = diabetes[['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age', 'BMI_log', 'feature_1', '0.0', '1.0', 'feature_3']]
Y = diabetes['Diabetes_012']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

# Standardizing the variables to help the support vector machine model
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [14]:
X_train

array([[0.        , 0.        , 0.23255814, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 1.        , 0.1744186 , ..., 0.        , 1.        ,
        1.        ],
       [0.        , 1.        , 0.15116279, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.6627907 , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.1744186 , ..., 0.        , 1.        ,
        0.6       ],
       [0.        , 0.        , 0.1627907 , ..., 1.        , 0.        ,
        0.        ]])

In [15]:
Y_train.value_counts()

0    170962
2     28277
1      3705
Name: Diabetes_012, dtype: int64

In [16]:
# Building Random Forest Classifier model
one_vs_all_RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

# Predicting on test dataset
one_vs_all_RF_pred = one_vs_all_RF.predict(X_test)

# Confusion Matrix
print(classification_report(Y_test, one_vs_all_RF_pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91     42741
           1       0.00      0.00      0.00       926
           2       0.00      0.00      0.00      7069

    accuracy                           0.84     50736
   macro avg       0.28      0.33      0.30     50736
weighted avg       0.71      0.84      0.77     50736



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [17]:
## OneVsOne strategy with logistic regression
one_vs_one_logit = OneVsOneClassifier(estimator = LogisticRegression()).fit(X_train, Y_train)

## Predicting on the test dataset
one_vs_one_logit_preds = one_vs_one_logit.predict(X_test)

## Computing the confusion matrix to evaluate the predictions
confusion_matrix(Y_test, one_vs_one_logit_preds)

NameError: name 'LogisticRegression' is not defined